# Problem 1.6

### Preambles
First, import packages needed and load datasets from files.

In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio

V = sio.loadmat('wordVecV.mat')['V']

As we can see the shape and summary of the data as follow:

In [2]:
print('Summary of V: \n', V)
print('Shape of V: ', V.shape)

Summary of V: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
Shape of V:  (1651, 10)


### (a)

Define the function for computing the minimum_distance pair:

In [3]:
from scipy.spatial import distance

def find_min_pairs(V, metric = 'euclidean'):
    
    # Compute distancematrix with repect to d:
    if metric == 'euclidean':
        dist_m= distance.cdist(V.T, V.T)
    elif metric == 'angle':
        # angle distance is equivalent to 'cosine' metric in scipy
        dist_m = distance.cdist(V.T, V.T, metric='cosine')
    else:
        raise ValueError("invalid metric!")
    
    # Make the distance matrix as triangled (zero-repalcing)
    # to leave unique pair;
    # Make the diagonal value cv vvas 0 to avoid computation error.
    dist_m = np.tril(dist_m)
    for i in range(len(dist_m)):
        dist_m[i, i] = 0.0
    
    # find the minimum value within each other and it's pair:
    min_dist = np.amin(dist_m[np.nonzero(dist_m)])
    min_index = np.where(dist_m == min_dist)
    
    return min_index

# Auxiliray function for print pairs

def print_pairs(pairs):
    for i in range(len(pairs[0])):        
        # given the index stated from 1
        print("Article No.%d and Articale No.%d." % (pairs[0][i] + 1, pairs[1][i] + 1))


Where the order of angle distance is euqivalent to 'cosine' metric of Scipy package, since it computes distance as:

$$1 - \frac{u\cdot v}{\left \| u \right \|_{2}\left \| v \right \|_{2}}$$

which the equivalence can be proved with the mononically deceasing of arccos function

In [4]:
min_pairs_euclid = find_min_pairs(V, metric = 'euclidean')
min_pairs_angle = find_min_pairs(V, metric = 'angle')

# print all posible distance pairs (redundent) of minimum index
print("All of posible euclid_minimum pairs: ")
print_pairs(min_pairs_euclid)
print()
# print all posible distance pairs (redundent) of minimum index
print("All pairs of posible angle_minimum pairs: ")
print_pairs(min_pairs_angle)

All of posible euclid_minimum pairs: 
Article No.8 and Articale No.7.

All pairs of posible angle_minimum pairs: 
Article No.10 and Articale No.9.


Thus, smallest distance pair is not same as samllest angle pair. The reason why is that the nearest distance neighbour and nearest angle neighbour of a vector to a vector set can be different, according to problem1.5.

### (b)

Now just create the normalized vectors set.

In [5]:
V_n = V.copy()
V_n = V_n / np.sum(V_n, axis=0)

Find cloest pair with normalized vectors.

In [6]:
min_pairs_euclid_n = find_min_pairs(V_n, metric = 'euclidean')
min_pairs_angle_n = find_min_pairs(V_n, metric = 'angle')

# print all posible distance pairs (redundent) of minimum index
print("All of posible euclid_minimum pairs with normalization: ")
print_pairs(min_pairs_euclid_n)
print()
# print all posible distance pairs (redundent) of minimum index
print("All of posible angle_minimum pairs with normalization: ")
print_pairs(min_pairs_angle_n)

All of posible euclid_minimum pairs with normalization: 
Article No.10 and Articale No.9.

All of posible angle_minimum pairs with normalization: 
Article No.10 and Articale No.9.


As we can see, angle closest pair and euclid closest pair is the same. Because all the normalized vectors has normalized. 

To use this kind of normalization can lead to a easier objective results.

### (c)


Compute the $w(t,d)$ and excute the minimum pair search:

In [7]:
doc_freq = (V > 0).sum(axis = 1)
w = V_n * np.sqrt(np.log(10 / doc_freq)).reshape(-1, 1)

min_pairs_euclid_w = find_min_pairs(w, metric = 'euclidean')

print("Posible pairs of closest TF-IDF: ")
print_pairs(min_pairs_euclid_w)

Posible pairs of closest TF-IDF: 
Article No.10 and Articale No.9.


### (d)

The higher *inverse document frequency* we get, the less frequency of a specific word that exist in entire document, which shows the word has more ability to identify different articles. 

Geometrically, Vectors that have more similarity (in number) with a specific word will become closer in vector space.